# Create platemap figures based on the platemap files

## Import libraries

In [1]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(platetools))

Warning message:
“package ‘ggplot2’ was built under R version 4.2.3”
Warning message:
“package ‘platetools’ was built under R version 4.2.3”


# Set paths to each platemap and output

In [2]:
platemap_files <- list.files(pattern = "^platemap_NF1*", full.names = TRUE)
print(platemap_files)

output_fig_dir <- file.path("platemap_figures")
platemap_suffix <- "_platemap_figure.png"

output_platemap_files <- list()
for (platemap_file in platemap_files) {
    # Extract plate name and remove suffix 
    plate <- basename(platemap_file)
    plate <- stringr::str_remove(plate, "_platemap.csv")
    plate <- stringr::str_extract(plate, "(?<=platemap_NF1_).*")  # Extracts the plate name
    plate <- stringr::str_remove(plate, "\\.csv$")  # Remove the .csv extension

    output_platemap_files[[plate]] <- file.path(output_fig_dir, paste0(plate, platemap_suffix))
}

print(output_platemap_files)


[1] "./platemap_NF1_plate1.csv" "./platemap_NF1_plate2.csv"
[3] "./platemap_NF1_plate3.csv" "./platemap_NF1_plate4.csv"
[5] "./platemap_NF1_plate5.csv" "./platemap_NF1_plate6.csv"
$plate1
[1] "platemap_figures/plate1_platemap_figure.png"

$plate2
[1] "platemap_figures/plate2_platemap_figure.png"

$plate3
[1] "platemap_figures/plate3_platemap_figure.png"

$plate4
[1] "platemap_figures/plate4_platemap_figure.png"

$plate5
[1] "platemap_figures/plate5_platemap_figure.png"

$plate6
[1] "platemap_figures/plate6_platemap_figure.png"



## Load in each platemap CSV for each plate

In [3]:
# Load in all platemap CSV files
platemap_dfs <- list()
for (plate in names(output_platemap_files)) {
    # Find the umap file associated with the plate
    platemap_file <- platemap_files[stringr::str_detect(platemap_files, plate)]
    
    # Load in the umap data
    df <- readr::read_csv(
    platemap_file,
    col_types = readr::cols(.default = "c")
)

    platemap_dfs[[plate]] <- df 
}

print(platemap_dfs)

$plate1
# A tibble: 8 × 5
  WellRow WellCol well_position gene_name genotype
  <chr>   <chr>   <chr>         <chr>     <chr>   
1 C       6       C6            NF1       WT      
2 C       7       C7            NF1       Null    
3 D       6       D6            NF1       WT      
4 D       7       D7            NF1       Null    
5 E       6       E6            NF1       WT      
6 E       7       E7            NF1       Null    
7 F       6       F6            NF1       WT      
8 F       7       F7            NF1       Null    

$plate2
# A tibble: 32 × 5
   WellRow WellCol well_position gene_name genotype
   <chr>   <chr>   <chr>         <chr>     <chr>   
 1 A       1       A1            NF1       WT      
 2 A       6       A6            NF1       WT      
 3 A       7       A7            NF1       Null    
 4 A       12      A12           NF1       Null    
 5 B       1       B1            NF1       WT      
 6 B       6       B6            NF1       WT      
 7 B       7       B

## Create platemap figures for only plates 1 and 2

In [4]:
for (plate in names(platemap_dfs)) {
    if (plate %in% c("plate1", "plate2")) {
    # Get the updated plate name
    updated_plate <- paste0("Plate ", as.numeric(gsub("plate", "", plate)))

    # output for each plate
    output_file <- output_platemap_files[[plate]]
    output_file <- paste0(output_file)
    
    platemap <-
        platetools::raw_map(
            data = platemap_dfs[[plate]]$genotype,
            well = platemap_dfs[[plate]]$well_position,
            plate = 96,
            size = 8
        ) +
        ggtitle(paste("Platemap layout for", updated_plate)) +
        theme(plot.title = element_text(size = 10, face = "bold")) +
        ggplot2::scale_fill_discrete(name = "Genotype") 

    ggsave(
        output_file,
        platemap,
        dpi = 500,
        height = 3.5,
        width = 6
    )
    }
}

Warning message:
“Warning: Your well label count ( 8 ) covers less than half the selected plate( 96 ).
Are you sure argument 'plate' is correct for the number of wells in your data?
note: Default is a 96-well plate.”
Warning message:
“Warning: Your well label count ( 32 ) covers less than half the selected plate( 96 ).
Are you sure argument 'plate' is correct for the number of wells in your data?
note: Default is a 96-well plate.”


## Create platemap figure for only plate 3

In [5]:
for (plate in names(platemap_dfs)) {
    if (plate %in% c("plate3")) {
    # Get the updated plate name
    updated_plate <- paste0("Plate ", as.numeric(gsub("plate", "", plate)))

    # output for each plate
    output_file <- output_platemap_files[[plate]]
    output_file <- paste0(output_file)
    
    platemap <-
        platetools::raw_map(
            data = as.numeric(platemap_dfs[[plate]]$seed_density),
            well = platemap_dfs[[plate]]$well_position,
            plate = 96,
            size = 8
        ) +
        ggtitle(paste("Platemap layout for", updated_plate)) +
        theme(plot.title = element_text(size = 10, face = "bold")) +
        ggplot2::geom_point(aes(shape = platemap_dfs[[plate]]$genotype)) +
        ggplot2::scale_shape_discrete(name = "Genotype") +
        ggplot2::scale_fill_gradient2(
        name = "Seed Density",
        low = "white",
        high = "red",
        )  

    ggsave(
        output_file,
        platemap,
        dpi = 500,
        height = 3.5,
        width = 6
    )
    }
}

## Create platemap figure for only plate 4

In [6]:
for (plate in names(platemap_dfs)) {
    if (plate %in% c("plate4")) {
    # Get the updated plate name
    updated_plate <- paste0("Plate ", as.numeric(gsub("plate", "", plate)))

    # Remove .png extension and add new suffixes
    filename_without_ext <- tools::file_path_sans_ext(output_platemap_files[[plate]])
    output_file_genotype <- paste0(filename_without_ext, "_genotype.png")
    output_file_dose <- paste0(filename_without_ext, "_dose.png")

    # Platemap for genotype
    platemap_genotype <- platetools::raw_map(
        data = platemap_dfs[[plate]]$genotype,
        well = platemap_dfs[[plate]]$well_position,
        plate = 96,
        size = 8
    ) +
    ggtitle(paste("Genotype platemap layout for", updated_plate)) +
    theme(plot.title = element_text(size = 10, face = "bold")) +
    ggplot2::scale_fill_discrete(name = "Genotype")
    
    # Platemap for dose
    platemap_dose <- platetools::raw_map(
        data = as.character(platemap_dfs[[plate]]$Concentration),
        well = platemap_dfs[[plate]]$well_position,
        plate = 96,
        size = 8
    ) +
    ggtitle(paste("siRNA treatment and dose platemap layout for", updated_plate)) +
    theme(plot.title = element_text(size = 10, face = "bold")) +
    ggplot2::geom_point(aes(shape = platemap_dfs[[plate]]$siRNA)) +
    ggplot2::scale_shape_discrete(name = "siRNA Treatments", limits = c("None", "Scramble", "NF1 Target 1", "NF1 Target 2")) +
    ggplot2::scale_fill_manual(
        name = "Concentrations (nM)",
        values = c("#eff6ef", "#b7deb7", "#56bc56", "#018301", "#015201", "#003800"),
        guide = guide_legend(override.aes = list(size = 5)),
    ) +
    theme(
        legend.text = element_text(size = 5),
        legend.title = element_text(size = 6),
        legend.position = "right",
        # move legend around so it fits better on the screen
        legend.margin = margin(-10, 0, 10, 0)
    )

    # Saving the platemaps
    ggsave(
        output_file_genotype,
        platemap_genotype,
        dpi = 500,
        height = 3.5,
        width = 6
    )
    
    ggsave(
        output_file_dose,
        platemap_dose,
        dpi = 500,
        height = 3.5,
        width = 6
    )
    }
}

## Create platemap figure for only plate 5

In [7]:
for (plate in names(platemap_dfs)) {
    if (plate %in% c("plate5")) {
    # Get the updated plate name
    updated_plate <- paste0("Plate ", as.numeric(gsub("plate", "", plate)))

    # output for each plate
    output_file <- output_platemap_files[[plate]]
    output_file <- paste0(output_file)
    
    platemap <-
        platetools::raw_map(
            data = platemap_dfs[[plate]]$genotype,
            well = platemap_dfs[[plate]]$well_position,
            plate = 96,
            size = 8
        ) +
        ggtitle(paste("Platemap layout for", updated_plate)) +
        theme(plot.title = element_text(size = 10, face = "bold")) +
        ggplot2::scale_fill_discrete(name = "Genotype")  

    ggsave(
        output_file,
        platemap,
        dpi = 500,
        height = 3.5,
        width = 6
    )
    }
}

## Create platemap figure for only plate 6

In [8]:
for (plate in names(platemap_dfs)) {
    if (plate == "plate6") {
        # Get the updated plate name
        updated_plate <- paste0("Plate ", as.numeric(gsub("plate", "", plate)))

        # Output for the plate
        output_file <- output_platemap_files[[plate]]
        output_file <- paste0(output_file)
        
        platemap <-
            platetools::raw_map(
                data = platemap_dfs[[plate]]$genotype,
                well = platemap_dfs[[plate]]$well_position,
                plate = 96,
                size = 8
            ) +
            ggtitle(paste("Platemap layout for", updated_plate)) +
            theme(plot.title = element_text(size = 10, face = "bold")) +
            ggplot2::scale_fill_discrete(name = "Genotype") +
            ggplot2::geom_point(aes(shape = platemap_dfs[[plate]]$Institution)) +
            ggplot2::scale_shape_discrete(name = "Institution\n(cell line)")

        ggsave(
            output_file,
            platemap,
            dpi = 500,
            height = 3.5,
            width = 6
        )
    }
}